## Question 4

In [1]:
pip install -r requirements.txt


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score

In [4]:
## Load and clean data

df_train = pd.read_csv("train.csv")
df_train = df_train.drop(columns=["zipcode"])

columns = df_train.drop(columns=["price"]).columns.tolist()
df_train[columns] = (df_train[columns] - df_train[columns].mean()) / df_train[columns].std()
df_train["price"] = df_train["price"] / 1000

df_test = pd.read_csv("test.csv")
df_test = df_test.drop(columns=["zipcode", "id", "date"])

columns = df_test.drop(columns=["price"]).columns.tolist()
df_test[columns] = (df_test[columns] - df_test[columns].mean()) / df_test[columns].std()
df_test["price"] = df_test["price"] / 1000

In [5]:
## Define custom linear regression class used closed form

class LinearRegression:

    def __init__(self):
        self.beta = None

    def fit(self, X, y):
        ones = np.ones((X.shape[0], 1))
        X_b = np.hstack((ones, X))
        
        self.beta = np.linalg.inv(X_b.T @ X_b) @ X_b.T @ y
        
    def predict(self, X):
        ones = np.ones((X.shape[0], 1))
        X_b = np.hstack((ones, X))
        return X_b @ self.beta

In [6]:
## Define polynomial expansion function

def poly_expansion(X, p):
    X_poly = np.column_stack([X**i for i in range(1, p+1)])
    return X_poly

In [7]:
## Use only sqft_living feature

X_train = df_train["sqft_living"]
y_train = df_train["price"]

X_test = df_test["sqft_living"]
y_test = df_test["price"]

In [14]:
## Train models with degrees 1 <= p <= 5

degrees = [1, 2, 3, 4, 5]

results = [('p', 'Train MSE', 'Test MSE', 'Train R^2', 'Test R^2')]

for p in degrees:
    
    X_train_poly = poly_expansion(X_train, p)
    X_test_poly = poly_expansion(X_test, p)
    
    model = LinearRegression()
    model.fit(X_train_poly, y_train)
    
    y_train_pred = model.predict(X_train_poly)
    y_test_pred = model.predict(X_test_poly)
    
    train_mse = mean_squared_error(y_train, y_train_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)
    
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    
    results.append((p, train_mse, test_mse, train_r2, test_r2))


## Print all results
for r in results:
    print(r)

('p', 'Train MSE', 'Test MSE', 'Train R^2', 'Test R^2')
(1, 57947.52616128836, 89911.92389151039, 0.49670880166311404, 0.460723507408658)
(2, 54822.665116276665, 74029.41541851572, 0.5238491329967204, 0.5559841034693498)
(3, 53785.19471649393, 96405.66370869218, 0.5328598665920146, 0.4217751557235575)
(4, 52795.77475764029, 185910.07513220602, 0.5414532680663737, -0.11505714609845352)
(5, 52626.11195455666, 336120.77136267413, 0.5429268390906048, -1.0159954633633892)


Looking at the MSE and R^2 for both the training data and the testing data for p values 1 through 5, we can see that for the training data, the MSE decreases and the R^2 value increases with each increase of p. This shows that for every added dimension of polynomial, the model fits the training data slightly better. However, when looking at the testing MSE and R^2, we see that it improves until p=2, and then starts to get significantly worse. at p=3, the MSE increases dramatically and the R^2 value goes lower than it was at p=1. This tells us that the optimal degree of polynomial using sqft_living as a feature to get the best predictions from the test data is p=2.